In [1]:
#installing ammadeus library
pip install amadeus

Note: you may need to restart the kernel to use updated packages.


In [2]:
#dotenv to hide api credentials
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
#modules from respective library
from amadeus import Client, ResponseError
import pandas as pd
import json

In [10]:
from dotenv import load_dotenv
import os

In [12]:
#pulling the data
load_dotenv()
amadeus = Client(
    client_id= os.getenv('amdeus_id'),
    client_secret= os.getenv('amadeus_secr') 
)

In [13]:
try: 
    response = amadeus.airport.direct_destinations.get(
                departureAirportCode='PTY')
    print(response.data)
except ResponseError as error:
    print(error)

[{'type': 'location', 'subtype': 'city', 'name': 'SAN ANDRES ISLAND', 'iataCode': 'ADZ', 'geoCode': {'latitude': 12.58362, 'longitude': -81.71111}, 'address': {'countryName': 'COLOMBIA', 'countryCode': 'CO', 'regionCode': 'SAMER'}, 'timeZone': {'offSet': '-05:00', 'referenceLocalDateTime': '2023-06-07T22:22:31'}}, {'type': 'location', 'subtype': 'city', 'name': 'AMSTERDAM', 'iataCode': 'AMS', 'geoCode': {'latitude': 52.31028, 'longitude': 4.76028}, 'address': {'countryName': 'NETHERLANDS', 'countryCode': 'NL', 'regionCode': 'EUROP'}, 'timeZone': {'offSet': '+02:00', 'referenceLocalDateTime': '2023-06-07T22:22:31'}}, {'type': 'location', 'subtype': 'city', 'name': 'ASUNCION', 'iataCode': 'ASU', 'geoCode': {'latitude': -25.23972, 'longitude': -57.51916}, 'address': {'countryName': 'PARAGUAY', 'countryCode': 'PY', 'regionCode': 'SAMER'}, 'timeZone': {'offSet': '-04:00', 'referenceLocalDateTime': '2023-06-07T22:22:31'}}, {'type': 'location', 'subtype': 'city', 'name': 'ATLANTA', 'iataCode'

In [ ]:
#checking variable type
destinations = response.data
print(type(destinations))

In [ ]:
#transforming to json
json = json.dumps(destinations)

In [ ]:
#reading data using pandas
datos = pd.read_json(json, orient = 'records')
datos

In [ ]:
#checking for null values 
datos.isnull().values.any()

In [ ]:
#renaming columns
for column in datos:
    datos = datos.rename(columns = {column : column.capitalize()})
datos = datos.rename(columns = {'Iatacode' : 'IATA Code'})
datos

In [ ]:
#function to extract desire data
def extract(iterable, key_1, key_2):
    keys = {}
    countries = []
    for a in iterable:
        keys = a.get(key_1)
        countries.append(keys[key_2])
    return countries

In [ ]:
#extracting desire data
countries = extract(destinations, 'address', 'countryName')
latitude = extract(destinations, 'geoCode', 'latitude')
longitude = extract(destinations, 'geoCode', 'longitude')


In [ ]:
#changing columns name
datos['Country'], datos['Latitude'], datos['Longitude'] = [countries, latitude, longitude]


In [ ]:
#clenaing not desire data 
datos.drop(datos.columns[[0, 1, 4, 5, 6]], axis = 1, inplace = True)

In [ ]:
#rearenging columns order
cols = ['Name', 'Country', 'IATA Code', 'Latitude', 'Longitude']
datos = datos[cols]
datos

In [ ]:
#creating a csv file 
datos.to_csv('./projects/Airport_routes.csv', index = False)